# FLIP (04): Learning Theory (I)
**(Module 03: Operation Research)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, but NOT allowed to change or distribute this package.

Prepared by and for 
**Student Members** |
2006-2018 [TULIP Lab](http://www.tulip.org.au), Australia

---

# Transportation Networks

This [IPython notebook](http://ipython.org/notebook.html) demonstrates the solution of transportation network problems using GLPK/MathProg. Problem data is adapted from Chapter 5 of Johannes Bisschop, "AIMMS Optimization Modeling", Paragon Decision Sciences, 1999. 

### Initializations

In [ ]:
from IPython.core.display import HTML
HTML(open("styles/custom.css", "r").read())

## Background

The prototypical transportation problem deals with the distribution of a commodity from a set of sources to a set of destinations. The object is to minimize total transportation costs while satisfying constraints on the supplies available at each of the sources, and satisfying demand requirements at each of the destinations.

Here we illustrate the transportation problem using an example from Chapter 5 of Johannes Bisschop, "AIMMS Optimization Modeling", Paragon Decision Sciences, 1999. In this example there are two factories and six customer sites located in 8 European cities as shown in the following map. The customer sites are labeled in red, the factories are labeled in blue.

<img src = "img/TransportationNetworksMap.png"  style="width:1000px"/>

Transportation costs between sources and destinations are given in units of &euro;/ton of goods shipped, and list in the following table along with source capacity and demand requirements.

#### Table of Transportation Costs, Customer Demand, and Available Supplies

| Customer\Source | Arnhem [&euro;/ton] | Gouda [&euro;/ton] | Demand [tons]|
| :--: | :----: | :---: | :----: |
| London | n/a | 2.5 | 125 |
| Berlin | 2.5 | n/a | 175 |
| Maastricht | 1.6 | 2.0 | 225 |
| Amsterdam | 1.4 | 1.0 | 250 |
| Utrecht | 0.8 | 1.0 | 225 |
| The Hague | 1.4 | 0.8 | 200 |
| **Supply [tons]** | 550 tons | 700 tons |  |

The situation can be modeled by links connecting a set nodes representing sources to a set of nodes representing customers.

<img src="img/TransportNet.png" style="height:400px">

For each link we can have a parameter $T[c,s]$ denoting the cost of shipping a ton of goods over the link. What we need to determine is the amount of goods to be shipped over each link, which we will represent as a non-negative decision variable $x[c,s]$.

The problem objective is to minimize the total shipping cost to all customers from all sources. 

$$\mbox{minimize:}\quad \mbox{Cost} = \sum_{c \in Customers}\sum_{s \in Sources} T[c,s] x[c,s]$$

Shipments from all sources can not exceed the manufacturing capacity of the source.

$$\sum_{c \in Customers} x[c,s] \leq \mbox{Supply}[s] \qquad \forall s \in Sources$$

Shipments to each customer must satisfy their demand.

$$\sum_{s\in Sources} x[c,s] = \mbox{Demand}[c] \qquad \forall c \in Customers$$

## MathProg Model

### Model File

In [ ]:
%%writefile TransportNet.mod

# Step 1: Define the sets
set SOURCES;
set CUSTOMERS;

# Step 2: Define parameters. Note the use of checks and defaults values.
param Demand {CUSTOMERS} >= 0;
param Supply {SOURCES}   >= 0;
param T {CUSTOMERS, SOURCES} default 1000;

# Step 3: Define the decision variables
var x {CUSTOMERS, SOURCES} >= 0;

# Step 4: Write the objective
minimize Cost: sum{c in CUSTOMERS, s in SOURCES} T[c,s]*x[c,s];

# Step 5: Write the constraints
subject to SRC {s in SOURCES}: sum {c in CUSTOMERS} x[c,s] <= Supply[s];
subject to DST {c in CUSTOMERS}: sum {s in SOURCES} x[c,s] = Demand[c];

# Step 6: Call for a solution
solve;

# Step 7: Write results to .csv files
printf "\nTotal Cost = %7.2f\n\n", Cost;

table shiptable {c in CUSTOMERS, s in SOURCES} OUT "CSV" "ship.csv" :
    c~Customer, s~Source, x[c,s]~Quantity, x[c,s].dual~Marginal, T[c,s]*x[c,s]~Cost;

table sources {s in SOURCES} OUT "CSV" "source.csv" :
    s~Source, Supply[s]~Supply, SRC[s]~Activity, SRC[s].dual~Marginal;
    
table customers {c in CUSTOMERS} OUT "CSV" "customer.csv" :
    c~Customer, Demand[c]~Demand, DST[c]~Activity, DST[c].dual~Marginal;

end;



### Data File

In [ ]:
%%script glpsol -m TransportNet.mod -d /dev/stdin -o output.txt

/* 
Problem Data from Chapter 5 of Johannes Bisschop, "AIMMS Optimization Modeling",
Paragon Decision Sciences, 1999. 
*/

param: CUSTOMERS: Demand :=
   Lon   125        # London
   Ber   175        # Berlin
   Maa   225        # Maastricht
   Ams   250        # Amsterdam
   Utr   225        # Utrecht
   Hag   200 ;      # The Hague

param: SOURCES: Supply :=
   Arn   550        # Arnhem
   Gou   700 ;      # Gouda

param T : Arn   Gou :=
   Lon        .    2.5
   Ber       2.5    .
   Maa       1.6   2.0
   Ams       1.4   1.0
   Utr       0.8   1.0
   Hag       1.4   0.8 ;

end;

## Solution

In [ ]:
import pandas as pd
df = pd.read_csv('ship.csv')
df[df['Quantity'] > 0]

The solution has the interesting property that, with the exception of Utrecht, customers are served by just one source.

<img src="img/TransportNet_soln.png" style="height:400px">


## Sensitivity Analysis

#### Analysis by Source

In [ ]:
import pandas as pd
pd.read_csv('source.csv')

The 'marginal' values are telling us how much the total costs will be increased for each one ton increase in the available supply from each source. The optimization calculation says that only 650 tons of the 700 available from Gouda should used for a minimum cost solution, which rules out any further cost reductions by increasing the available supply. In fact, we could decrease the supply Gouda without any harm. The marginal value of Gouda is 0.

The source at Arnhem is a different matter. First, all 550 available tons are being used. Second, from the marginal value we see that the total transportations costs would be reduced by 0.2 Euros for each additional ton of supply.  

The management conclusion we can draw is that there is excess supply available at Gouda which should, if feasible, me moved to Arnhem.

Now that's a valuable piece of information!

#### Analysis by Customer

In [ ]:
pd.read_csv('customer.csv')

Looking at the demand constraints, we see that all of the required demands have been met by the optimal solution.

The marginal values of these constraints indicate how much the total transportation costs will increase if there is an additional ton of demand at any of the locations. In particular, note that increasing the demand at Berlin will increase costs by 2.7 Euros per ton. This is actually **greater** than the list price for shipping to Berlin which is 2.5 Euros per ton.  Why is this?

To see what's going on, let's resolve the problem with a one ton increase in the demand at Berlin.

In [ ]:
%%script glpsol -m TransportNet.mod -d /dev/stdin -o output.txt

/* 
Problem Data from Chapter 5 of Johannes Bisschop, "AIMMS Optimization Modeling",
Paragon Decision Sciences, 1999. 
*/

param: CUSTOMERS: Demand :=
   Lon   125        # London
   Ber   176        # Berlin      <=== INCREASE BY ONE TON
   Maa   225        # Maastricht
   Ams   250        # Amsterdam
   Utr   225        # Utrecht
   Hag   200 ;      # The Hague

param: SOURCES: Supply :=
   Arn   550        # Arnhem
   Gou   700 ;      # Gouda

param T : Arn   Gou :=
   Lon        .    2.5
   Ber       2.5    .
   Maa       1.6   2.0
   Ams       1.4   1.0
   Utr       0.8   1.0
   Hag       1.4   0.8 ;

end;

We see the total cost has increased from 1715.0 to 1717.7 Euros, an increase of 2.7 Euros just as predicted by the marginal value assocated with the demand constraint for Berlin.

Now let's look at the solution.

In [ ]:
df = pd.read_csv('ship.csv')
df[df['Quantity'] > 0]

Here we see that increasing the demand in Berlin resulted in a number of other changes. This figure shows the changes shipments.

<img src="img/TransportNet_sens.png" style="height:400px">

* Shipments to Berlin increased from 175 to 175 tons, increasing costs for that link from 437.5 to 440.0, or a net increase of 2.5 Euros.
* Since Arnhem is operating at full capacity, increasing the shipments from Arnhem to Berlin resulted in decreasing the shipments from Arhhem to Utrecht from 150 to 149 reducing those shipping costs from 120.0 to 119.2, a net decrease of 0.8 Euros.
* To meet demand at Utrecht, shipments from Gouda to Utrecht had to increase from 75 to 76, increasing shipping costs by a net amount of 1.0 Euros.
* The net effect on shipping costs is 2.5 - 0.8 + 1.0 = 2.7 Euros.

The important conclusion to draw is that when operating under optimal conditions, a change in demand or supply can have a ripple effect on the optimal solution that can only be measured through a proper sensitivity analysis.

### Exercises

1. Move 50 tons of supply capacity from Gouda to Arnhem, and repeat the sensitivity analysis. How has the situation improved?  In practice, would you recommend this change, or would you propose something different?
2. What other business improvements would you recommend?